In [ ]:
import os
import json

import pprint as pp

import gymnasium as gym
import logic_gym
import graphviz

MAX_THEOREMS = 100
ONE_HOP_THEOREMS_FOLDER = "dataprep"
ONE_HOP_THEOREMS = "1hop_trueontology.json"


class TheoremsManipulator:
    def __init__(self):
        self.tasks = {}
        self.english_theorems = {}
        self.file_path = os.path.join(ONE_HOP_THEOREMS_FOLDER, ONE_HOP_THEOREMS)
        with open(self.file_path, "r") as file:
            json_theorems = json.load(file)
            self.json_theorems = json_theorems

            for key in json_theorems:
                # print(json_theorems[key]['test_example'])
                english_theorems = json_theorems[key]['test_example']["question"]
                english_queries = json_theorems[key]['test_example']["query"]
                self.english_theorems[key] = english_theorems.split(". ")
                self.english_theorems[key].extend(english_queries.split(". "))
                
                if "formalized" in json_theorems[key]['test_example']:
                    self.tasks[key] = (json_theorems[key]['test_example']['formalized'])
                    

    def get_theorem(self, theorem = 1):
        if f"example{theorem}" in self.tasks:
            return self.tasks[f"example{theorem}"]
        else:
            return None
        
    def get_english_theorem(self, theorem = 1):
        if f"example{theorem}" in self.english_theorems:
            return self.english_theorems[f"example{theorem}"]
        else:
            return None

    def set_proved(self, theorem = 1, proved = False):
        self.json_theorems[f"example{theorem}"]['test_example']["proved"] = proved
        
    def get_proved(self, theorem = 1):
        if "proved" in self.json_theorems[f"example{theorem}"]['test_example']:
            return self.json_theorems[f"example{theorem}"]['test_example']["proved"]
        return False
    
    
        
        
    def set_proof(self, theorem = 1, proof = None):
        self.json_theorems[f"example{theorem}"]['test_example']["proof"] = proof
    
    def write_back_updated_file(self):
        with open(self.file_path, "w") as file:
            json.dump(self.json_theorems, file, indent=4)



In [1]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os
import pprint as pp

#MODEL = "gpt-3.5-turbo"
MODEL = "gpt-4-turbo"


TEMPERATURE = 0.5

MAX_TOKENS = 2000


load_dotenv()  # take environment variables from .env.

client = OpenAI(
    api_key=os.getenv("OPEN_AI_API_KEY"),
)

# Specify the file path of the JSON file
#input_file_path = "1hop_trueontology.json"

file_path = "1hop_trueontology.json"

# Load the data from the JSON file
with open(file_path, "r") as file:
    data = json.load(file)
    
# with open(output_file_path, "r") as file:
#     output_data = json.load(file)

# Now you can use the 'data' variable as a Python data structure



In [2]:
#look at output_data and build a list of theorems not formalized yet

def find_theorems_not_unformalized(data, override = []):
    theorems_not_unformalized = []
    for i in range(len(data) - 1):
        if  "formalized" in data[f"example{i+1}"]["test_example"] and not "unformalized" in data[f"example{i+1}"]["test_example"]:
            theorems_not_unformalized.append(f"example{i+1}")
    for i in override:
        theorems_not_unformalized.append(f"example{i}")
    theorems_not_unformalized.sort()
    return theorems_not_unformalized

un_un_formalized = find_theorems_not_unformalized(data, override = [11,12,13,14,15])

pp.pprint(un_un_formalized)
type(un_un_formalized[0])



['example100',
 'example101',
 'example102',
 'example103',
 'example104',
 'example105',
 'example106',
 'example107',
 'example108',
 'example109',
 'example11',
 'example110',
 'example111',
 'example112',
 'example113',
 'example114',
 'example115',
 'example116',
 'example117',
 'example118',
 'example119',
 'example12',
 'example120',
 'example121',
 'example122',
 'example123',
 'example124',
 'example125',
 'example126',
 'example127',
 'example128',
 'example129',
 'example13',
 'example130',
 'example131',
 'example132',
 'example133',
 'example134',
 'example135',
 'example136',
 'example137',
 'example138',
 'example139',
 'example14',
 'example140',
 'example141',
 'example142',
 'example143',
 'example144',
 'example145',
 'example146',
 'example147',
 'example148',
 'example149',
 'example15',
 'example150',
 'example151',
 'example152',
 'example153',
 'example154',
 'example155',
 'example156',
 'example157',
 'example158',
 'example159',
 'example160',
 'example161',


str

In [3]:
if "example1" in un_un_formalized:
    print("yes")

In [4]:
with open("unformalize_prompt.txt", "r") as file:
    prompt_string = file.read()
    print(prompt_string)

Task Description: You are given a problem description and a question. The task is to: 
1) define all the predicates in the problem
2) parse the problem into logic rules based on the defined predicates
3) write all the facts mentioned in the problem, also identify all the variables
4) parse the question into the logic form as goals

Make sure not to generate English explanations at the end. 
Do not create markdown. 
Make sure to balance parenthesis. 
Make sure not to confuse variables and predicates

------
Problem:
Each jompus is fruity. Every jompus is a wumpus. Every wumpus is not transparent. Wumpuses are tumpuses. Tumpuses are mean. Tumpuses are vumpuses. Every vumpus is cold. Each vumpus is a yumpus. Yumpuses are orange. Yumpuses are numpuses. Numpuses are dull. Each numpus is a dumpus. Every dumpus is not shy. Impuses are shy. Dumpuses are rompuses. Each rompus is liquid. Rompuses are zumpuses. Alex is a tumpus.
Question:
True or false: Alex is not shy.
###

# Predicates
class Jo

In [5]:
CHATGPT_SYSTEM_MESSAGE = "You are an expert mathematician who can translate informal mathematical statements to formal statements and visa."


def get_input_chat_message(prompt):
    messages = [
        {"role": "system", "content": CHATGPT_SYSTEM_MESSAGE},
        {"role": "user", "content": prompt},
    ]
    return messages


def get_formalized_from_GPT(client, messages):
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
    )
    return response.choices[0].message.content

def get_flip_formalized_version(formalized):
    return formalized

def get_formalized_version(prompt_string):
    # Call the GPT API and get the formalized output
    # For now, we will just return a dummy response
    
    message = get_input_chat_message(prompt_string)
    formalized = get_formalized_from_GPT(client, message)
    flip_formalized = get_flip_formalized_version(formalized)
    
    return flip_formalized


In [ ]:
from time import sleep


MAX_PROBLEMS_TO_PROCESS = 100

index = 0
for key, value in data.items():
    if not key in un_un_formalized:
        continue
    if index >= MAX_PROBLEMS_TO_PROCESS:
        break
    index += 1
    print("Formalizing ", key)
    for key2, value2 in value.items():
        if key2 == "test_example":
            name, question, query = key, value2["question"], value2["query"]
            # Read the contents of the prompt.txt file
            with open("prompt.txt", "r") as file:
                prompt_string = file.read()

            # Replace [[PROBLEM]] with the value of 'contents'
            prompt_string = prompt_string.replace("[[PROBLEM]]", question)

            # Replace [[Question]] with the value of 'question'
            prompt_string = prompt_string.replace("[[Question]]", query)

            # Print the modified prompt string
            formalized = get_formalized_version(prompt_string)
            value2["formalized"] = formalized
            sleep(30)
            with open(file_path, "w") as file:
                json.dump(data, file, indent=4)
            # print(formalized)
            


# Save the data as JSON
with open(file_path, "w") as file:
    json.dump(data, file, indent=4)

Formalizing  example11
Formalizing  example12
Formalizing  example13
Formalizing  example14
Formalizing  example15
Formalizing  example21
Formalizing  example22
Formalizing  example23
Formalizing  example24
Formalizing  example25
Formalizing  example26
Formalizing  example27
Formalizing  example28
Formalizing  example29
Formalizing  example30
Formalizing  example31
Formalizing  example32
Formalizing  example33
Formalizing  example34
Formalizing  example35
Formalizing  example36
Formalizing  example37
Formalizing  example38
Formalizing  example39
Formalizing  example40
